#PRE-PROCESS DATA

##Importing Required Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import pickle


##Read the dataset

In [4]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='dD84UgrUINNoRZZmzo70HwuEPyMPKzIaFDIZK71HR1aw',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'modeldeployment-donotdelete-pr-f2uff7wxmkxxwi'
object_key = 'autos_preprocessed.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

newdf = pd.read_csv(body)
newdf.head()


# import os, types
# from botocore.client import Config
# import ibm_boto3

# def __iter__(self): return 0

# # @hidden_cell
# # The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# # You might want to remove those credentials before you share the notebook.
# cos_client = ibm_boto3.client(service_name='s3',
#     ibm_api_key_id='dD84UgrUINNoRZZmzo70HwuEPyMPKzIaFDIZK71HR1aw',
#     ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
#     config=Config(signature_version='oauth'),
#     endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

# bucket = 'modeldeployment-donotdelete-pr-f2uff7wxmkxxwi'
# object_key = 'autos.csv'

# body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# # add missing __iter__ method, so pandas accepts body as file-like object
# if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# df = pd.read_csv(body)
# df.head()
# df = pd.read_csv("autos.csv", header=0, encoding="Latin1")
# df.head()

,Unnamed: 0,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,1,18300,coupe,2011,manual,190.0,not-declared,125000.0,5.0,diesel,audi,yes
1,2,9800,suv,2004,automatic,163.0,grand,125000.0,8.0,diesel,jeep,not-declared
2,3,1500,small car,2001,manual,75.0,golf,150000.0,6.0,petrol,volkswagen,no
3,4,3600,small car,2008,manual,69.0,fabia,90000.0,7.0,diesel,skoda,no
4,5,650,limousine,1995,manual,102.0,3er,150000.0,10.0,petrol,bmw,yes


##Cleaning dataset

###Printing different sellers

In [ ]:
print(df.seller.value_counts())

###Removing the seller "gewerblich"

In [ ]:
df[df.seller != "gewerblich"]

###Dropping the column seller, as all the entries are same

In [ ]:
df = df.drop("seller", 1)

###Printing different offer type

In [ ]:
print(df.offerType.value_counts())

###Dropping the offer type "Gesuch"

In [ ]:
df[df.offerType != "Gesuch"]

###Dropping the column offerType since it has same entries

In [ ]:
df = df.drop("offerType", 1)
print(df.shape)

###Removing cars having power less than 50hp and greater than 900hp

In [ ]:
df = df[(df.powerPS > 50) & (df.powerPS < 900)]
print(df.shape)

###Keeping only the cars which are registered between 1950 and 2017

In [ ]:
df = df[(df.yearOfRegistration >= 1950) & (df.yearOfRegistration < 2017)]
print(df.shape)

###Removing irrelevant columns

In [ ]:
df.drop(["name", "abtest", "dateCrawled", "nrOfPictures", "lastSeen", "postalCode", "dateCreated"], axis = "columns", inplace = True)

###Dropping the duplicates in the dataframe and storing it in a new dataframe

In [ ]:
newdf = df.copy()
newdf = newdf.drop_duplicates(["price", "vehicleType", "yearOfRegistration", "gearbox", "powerPS", "model", "kilometer", "monthOfRegistration", "fuelType", "notRepairedDamage"])

###Replacing german words with proper english words

In [ ]:
newdf.gearbox.replace(("manuell","automatik"), ("manual", "automatic"), inplace = True)
newdf.fuelType.replace(("benzin","andere","elektro"), ("petrol", "others", "electric"), inplace = True)
newdf.vehicleType.replace(("kleinwagen","cabrio","kombi","andere"), ("small car","convertible","combination",
                            "others"), inplace = True)
newdf.notRepairedDamage.replace(("ja","nein"), ("yes", "no"), inplace = True)

###Removing the outliners

In [ ]:
newdf = newdf[(newdf.price >= 100) & (newdf.price <= 150000)]

###Filling NaN

In [ ]:
newdf["notRepairedDamage"].fillna(value = "not-declared", inplace = True)
newdf["fuelType"].fillna(value = "not-declared", inplace = True)
newdf["gearbox"].fillna(value = "not-declared", inplace = True)
newdf["vehicleType"].fillna(value = "not-declared", inplace = True)
newdf["model"].fillna(value = "not-declared", inplace = True)

###Saving the cleaned dataset

In [ ]:
newdf.to_csv("/content/autos_preprocessed.csv")

###LabelEncoding the categorical data

In [5]:
labels = ["gearbox", "notRepairedDamage", "model", "brand", "fuelType", "vehicleType"]
mapping = {}
for i in labels:
    mapping[i] = LabelEncoder()
    mapping[i].fit(newdf[i])
    trans = mapping[i].transform(newdf[i])
    np.save(str("classes"+i+".npy"),mapping[i].classes_)
    print(i,":",mapping[i])
    newdf.loc[:,i+"_labels"] = pd.Series(trans, index = newdf.index)

labeled = newdf[["price", "yearOfRegistration", "powerPS", "kilometer", "monthOfRegistration"] + [x+"_labels" for x in labels]]
print(labeled.columns)

gearbox : LabelEncoder()
notRepairedDamage : LabelEncoder()
model : LabelEncoder()
brand : LabelEncoder()
fuelType : LabelEncoder()
vehicleType : LabelEncoder()
Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


##Splitting data into dependent and independent variables

###Sorting price

In [6]:
Y = labeled.iloc[:,0].values
X = labeled.iloc[:,1:].values
Y = Y.reshape(-1,1)

###Splitting the data into training and testing set

In [7]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 3)

##Choosing appropriate model

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
regressor = RandomForestRegressor(n_estimators=1000, max_depth=10, random_state=34)

##Fitting the model

In [9]:
regressor.fit(X_train, np.ravel(Y_train, order='C'))

RandomForestRegressor(max_depth=10, n_estimators=1000, random_state=34)

##Checking the metrics

In [10]:
y_pred = regressor.predict(X_test)

print(r2_score(Y_test, y_pred))

0.8150248765958094


##Saving model

In [ ]:
filename = "resale_model.sav"
pickle.dump(regressor, open(filename, "wb"))

In [11]:
!pip install ibm_watson_machine_learning

In [12]:
from ibm_watson_machine_learning import APIClient
wmlCred = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey": "JtQPfzFB5KgpVUWPiAstI6QMb6UkpOqhsZMARqEN6l-N"
}
client = APIClient(wmlCred)

In [13]:
def guide_from_space(client, space_name):
    space = client.spaces.get_details()
    return (next(item for item in space["resources"] if item["entity"]["name"] == space_name)["metadata"]["id"])

In [14]:
space_uid = guide_from_space(client, "Model Deployment")
print("Space UID: " + space_uid)

Space UID: 0068cb4e-c97d-49ae-b9cf-099b0d0cbd5c


In [15]:
client.set.default_space(space_uid)

'SUCCESS'

In [16]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [17]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [18]:
import sklearn
sklearn.__version__

'1.0.2'

In [19]:
# client.repository.ModelMetaNames.show() '154010fa-5b3b-4ac1-82af-4d5ee5abbc85' scikit-learn_0.23	default_py3.7_opence

# client.repository.ModelMetaNames.get()
metadata = {
     client.repository.ModelMetaNames.NAME: "Car price prediction",
     client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
     client.repository.ModelMetaNames.TYPE: "scikit-learn_1.0"
}


model_details = client.repository.store_model(model=regressor,meta_props=metadata, training_data=X_train, training_target=Y_train)
model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [22]:
model_id

'3f8e7ab7-2cbf-40d4-81cb-0e697a284b85'

In [23]:
X_train[0]

array([2.013e+03, 2.110e+02, 1.250e+05, 2.000e+00, 0.000e+00, 0.000e+00,
       9.100e+01, 1.800e+01, 1.000e+00, 8.000e+00])

In [ ]:
#regressor.predict()